# BACKTESTING STOCK MARKET SHORT PREDICTION MODEL

@author: Matheus José Oliveira dos Santos

## 1. Imports

In [1]:
import sys
import os
from datetime import date, datetime, timedelta
from typing import List

import pandas as pd
import numpy as np
import yfinance as yf
import joblib

sys.path.append(os.getcwd()+'\\..\\src')
import db_interface
import portfolio_backtest

## 2. class inheritance

In [2]:
class Backtest(portfolio_backtest.Backtest):
    def get_data(self) -> np.array:
        print("getting data")
        _ref_date = self._date_indicator
        table_name = "financials"
        
        my_db = db_interface.DB_interface("FINANCE_DB")

        with my_db:
            for i in range(5):
                #query only brazilian stocks
                query = "SELECT * FROM {0} WHERE date = '{1}'".format(table_name, _ref_date.strftime("%Y-%m-%d"))
                df_data = my_db.read_by_command(query)
                if df_data.shape[0] == 0:
                    #if there are no data, get the day before
                    _ref_date = _ref_date - timedelta(days=1)
                else:
                    break #exit loop
                    raise ValueError("There are not enough data")
        
        #pre-processing
        df_sample = df_data.copy().iloc[:,2:-3]
        sc = joblib.load(self._sc)
        ar_sample_scaled = sc.transform(df_sample)
        
        #save BBGTicker order
        self._order_BBGTicker = df_data['BBGTicker']

        #return the sample scaled and save the order of BBGTicker
        return ar_sample_scaled

    def allocation(self, df_input:pd.DataFrame) -> pd.DataFrame:
        #create base dataframe
        df_aloc = pd.DataFrame(columns=['BBGTicker','%AUM'])

        #add short position
        df_aloc['BBGTicker'] = df_input.tail(10)['BBGTicker']
        df_aloc['%AUM'] = -0.05

        #add long position
        aloc_aux = {'BBGTicker':'BOVA11', '%AUM':1.5}
        df_aloc = pd.concat([df_aloc, pd.DataFrame(aloc_aux,index=[0])], ignore_index=True)

        return df_aloc

    def return_securities(list_securities: List[str], week_start:date, week_end:date) -> pd.DataFrame:
        pass
        
    def new_year(self) -> None:
        pass

## 3. Backtest

In [3]:
start_date = date(2022,1,1)
end_date = date(2023,8,1)
delta = timedelta(weeks=1)

list_scalers = ['sc2013.pkl','sc2014.pkl','sc2015.pkl','sc2016.pkl','sc2017.pkl','sc2018.pkl','sc2019.pkl','sc2020.pkl','sc2021.pkl','sc2022.pkl','sc2023.pkl']
list_models = ['ANN2013','ANN2014','ANN2015','ANN2016','ANN2017','ANN2018','ANN2019','ANN2020','ANN2021','ANN2022','ANN2023']

In [4]:
bt = Backtest(start_date, end_date, delta)
bt.set_date_indicator(date(2023,7,21))
bt.set_scaler(os.getcwd()+'\\..\\scalers\\'+list_scalers[0])
ar_sample_sc = bt.get_data()
#bt.executar()

getting data
connecting in: FINANCE_DB


C:\Users\mathe\Coding Projects\quant-finance-strategy\notebooks\..\src\db_interface.py:98: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(command, self.db)


DB Closed


In [5]:
df_predictions = bt.make_predictions(model_path = os.getcwd()+'\\..\\models\\'+list_models[0], ar_sample = ar_sample_sc)
print(df_predictions.head(10))

4/4 [==============================] - 0s 0s/step
         BBGTicker  probability
0  WEGE3 BZ Equity     0.999985
1  GUAR3 BZ Equity     0.999900
2  CMIG3 BZ Equity     0.982663
3  SMTO3 BZ Equity     0.978808
4  RANI3 BZ Equity     0.276227
5  OIBR3 BZ Equity     0.031317
6  GFSA3 BZ Equity     0.013635
7  EMBR3 BZ Equity     0.010623
8  POSI3 BZ Equity     0.008574
9  PCAR3 BZ Equity     0.003797


In [6]:
df_aloc = bt.allocation(df_predictions)
print(df_aloc)

          BBGTicker  %AUM
0   MDIA3 BZ Equity -0.05
1   CSAN3 BZ Equity -0.05
2   TOTS3 BZ Equity -0.05
3   HYPE3 BZ Equity -0.05
4   ABEV3 BZ Equity -0.05
5   PTBL3 BZ Equity -0.05
6   COGN3 BZ Equity -0.05
7   NTCO3 BZ Equity -0.05
8   RADL3 BZ Equity -0.05
9   RENT3 BZ Equity -0.05
10           BOVA11  1.50


In [ ]:
def return_securities(list_securities: List[str], week_start:date, week_end:date) -> pd.DataFrame:
    #convert date to str
    
    #convert bloomberg ticker to yfinance ticker

    #get data and calculate the returns
    
    dict_returns = {}
    for stock in list_securities:
        data = yf.download(stock, week_start, week_end)
        ret_stock = data['Adj Close'][-1]/data['Adj Close'][0] - 1
        dict_returns[stock] = ret_stock

    df_returns = pd.DataFrame(dict_returns, columns = ['Ticker', 'Return'])

    return df_returns

list_test = ['KEPL3.SA','SMTO3.SA','VALE3.SA']
df = return_securities(list_test, date(2023,8,1),date(2023,9,1))
print(df.head())

Exception in thread Thread-5:
Traceback (most recent call last):
  File "c:\users\mathe\appdata\local\programs\python\python39\lib\site-packages\requests\models.py", line 971, in json
    return complexjson.loads(self.text, **kwargs)
  File "c:\users\mathe\appdata\local\programs\python\python39\lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "c:\users\mathe\appdata\local\programs\python\python39\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "c:\users\mathe\appdata\local\programs\python\python39\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\mathe\appdata\local\programs\python\python39\lib\threading.py", line 954, in _bootstrap_inner
 

In [ ]:
df = bt.get_performance()